In [1]:
# coding=utf8
import os
import numpy as np
import time
from langconv import *
import re
import sys
from collections import defaultdict

# pre-process

In [2]:
### 1 ### load vectors
tagName = []
tagVec = []
count = 0
for i in range(0,50):
    with open ('./vectors/part-000%02d'%i, 'r') as f:
        print('./vectors/part-000%02d'%i)
        for line in f.readlines():
            tag = line.strip().split('\t')[0]
            vec = line.strip().split('\t')[2].split(',')
            vec = np.array(vec, dtype=np.float64)
            tagVec.append(vec)
            tagName.append(tag)
            count += 1
            if count % 50000 == 0:
                print(count, tag)
                print(vec)

print(len(tagName),len(tagVec))
indexs = [tagName, tagVec]

./vectors/part-00000
5000 diy电脑
[-0.05582178  0.05021889 -0.05549474 -0.168147   -0.15853265 -0.0192858
 -0.06661185  0.13715096 -0.0700011  -0.01090996 -0.01617583 -0.00229601
 -0.00432137  0.0161366   0.01741199  0.01060888 -0.06434897 -0.13294066
 -0.11530998  0.01822936 -0.0073366   0.01651175 -0.0108033   0.03275665
  0.00917513  0.11818802 -0.07126089  0.04518494  0.01237248  0.03895342
  0.05148807  0.03949048  0.23572512 -0.01974818  0.10942902 -0.06137161
 -0.10010851  0.06575188 -0.03704993  0.0245985  -0.0141099   0.09067103
 -0.09644303 -0.07487897 -0.129998    0.03944759 -0.07066706  0.03019234
  0.03262683 -0.01514375  0.07572844  0.22438793  0.05126319  0.06097785
 -0.02298747  0.09663413 -0.01025819 -0.05959072 -0.02097039 -0.10237174
  0.0474041  -0.0303362   0.13038811  0.07738414 -0.02562917  0.15361162
 -0.09479382  0.04307733  0.02904569 -0.15459606  0.11000498 -0.02445814
  0.02070851  0.01454049 -0.06302337 -0.12746726 -0.01639728  0.0488241
 -0.01644033 -0.00619

./vectors/part-00004
35000 城市战场
[ 0.01567353  0.12456009 -0.01530206 -0.20898232 -0.05326318  0.01763882
  0.02681854 -0.08894907 -0.00135776  0.07994678 -0.0441829  -0.06656949
 -0.07326426 -0.10469124 -0.00661196  0.0365731  -0.05210733  0.04118248
 -0.04473602 -0.06524063  0.06485935 -0.0148145   0.03709816 -0.10006186
  0.0279799   0.19689715  0.11105306 -0.01469132  0.2381514   0.04034553
  0.02125643 -0.00808682 -0.00743595 -0.0561638  -0.02754977  0.11165565
 -0.00662741  0.05728519 -0.18161659 -0.04717181 -0.13917753 -0.08425142
 -0.0043756  -0.2356631   0.00531716  0.11409688 -0.0392203   0.10372526
  0.03365223  0.07594522  0.04889682 -0.07928932 -0.18622415  0.07513522
 -0.03960041 -0.13165356 -0.07773199  0.12170359 -0.27416578  0.04774695
 -0.13410293 -0.12672535  0.11618887 -0.02115147 -0.06018576 -0.08556783
 -0.02229291  0.00829487 -0.10118373 -0.09841948  0.04230148  0.09725673
  0.00365281 -0.0122113  -0.00416174  0.0388536   0.10356402  0.04121966
  0.05555035 -0.199

65000 韩式冷面
[ 0.02230501 -0.0880652  -0.00634085 -0.06178881 -0.03680226  0.09328566
 -0.05697006  0.08977372  0.01342255 -0.01999786 -0.11118528  0.11277331
 -0.11638143  0.01935864 -0.02501217  0.02182546  0.1959715  -0.00393092
 -0.18689665 -0.02317591 -0.03433544  0.03108807 -0.23678643  0.02093359
 -0.0256777  -0.0065988  -0.01521735  0.0319868   0.02726973  0.02102427
 -0.10440271 -0.00494097  0.0476897   0.02961925 -0.11298013 -0.08329251
  0.06785241 -0.0084534  -0.09144976 -0.04071223  0.05964174  0.10302145
  0.03382494 -0.0888598  -0.26992887 -0.02845594 -0.06895382  0.01253849
  0.03135594  0.06186997  0.05899497  0.11334234 -0.01446428  0.03687541
  0.04281718 -0.07459304  0.004962    0.13050216 -0.04912041 -0.04073784
 -0.20711143 -0.1100478   0.08079867  0.02012376 -0.11821125 -0.06016591
 -0.02355751 -0.00227023 -0.0625963   0.05852086 -0.09236609 -0.04774733
 -0.19141023  0.02744531  0.1132269   0.06373541  0.02640004  0.11752888
 -0.00233642  0.02376159 -0.02292573 -0.

./vectors/part-00011
95000 毛爸
[ 0.00645624 -0.25478175  0.24681242 -0.14127611 -0.13213779 -0.03454659
  0.13192311  0.09115213 -0.17067693  0.0954625  -0.19603835 -0.0036708
 -0.13118024 -0.11954928  0.02694239  0.09475737 -0.11976825 -0.14741062
  0.02612046  0.10258449 -0.01953611 -0.01776459 -0.13328512 -0.04429094
 -0.10373843 -0.09936462 -0.06664     0.12696931 -0.14105958  0.05793387
  0.00829471  0.02146965 -0.11195481  0.10885169  0.01116495 -0.04971459
  0.04300915 -0.02776738  0.09311517 -0.03735583  0.21679303  0.1898094
 -0.13740334 -0.06231202  0.03474658  0.23124143  0.03069622 -0.15708596
 -0.01809938 -0.06262945  0.02751465  0.01492353 -0.30223671 -0.04112358
 -0.2017256  -0.03986332 -0.2986199   0.24597542  0.0665561  -0.0894943
 -0.02939247 -0.12511441  0.07583061  0.16082299 -0.0824023  -0.02269866
 -0.09727602  0.0227563  -0.03601595 -0.08792928 -0.08893766  0.04801191
 -0.19329779 -0.08103444  0.1504858  -0.00870443  0.13687627 -0.06431355
 -0.05239492 -0.08731826

./vectors/part-00014
120000 bs日本のうた
[-0.14254072  0.0017465   0.04022746  0.17018101 -0.04782933 -0.20964761
 -0.02748287  0.00237795  0.124906   -0.20719402 -0.18010938  0.04507689
  0.06159181 -0.05307416 -0.41385049  0.2956703   0.06737708  0.24406275
 -0.53086364 -0.10607384  0.10320226  0.18746635  0.21159175 -0.02423644
 -0.25526735  0.2614271  -0.09295581  0.14399754  0.04453821  0.11724117
  0.15361273 -0.09364437  0.00937463 -0.21492283 -0.05539722  0.28722492
  0.25614247  0.10418732 -0.18821536 -0.14956631 -0.21955167 -0.19483331
 -0.04935391 -0.26612419  0.03350472 -0.14992511 -0.15573815  0.32489231
 -0.18707064 -0.01602183 -0.01585844 -0.12070331  0.05485003 -0.33529219
 -0.17002495 -0.07738122  0.0496076   0.2857601   0.02632321  0.1136864
  0.0750409  -0.00385116  0.09646255 -0.05025741  0.11897204  0.00759107
 -0.37043813 -0.06961758  0.1165321  -0.32230225  0.11755772  0.2870906
 -0.06103941 -0.10163234  0.08912043  0.00417154 -0.0487856   0.1521659
  0.05357109 -0.26

...

320000 90期
[ 0.12623075 -0.10265744 -0.0819983   0.06027436 -0.05075052 -0.11477013
 -0.13088335 -0.03711304 -0.14693536  0.24188052  0.01985825 -0.05748476
 -0.13503839 -0.16999191 -0.07673702  0.11511882  0.08568701  0.01713339
  0.01217925  0.12871523  0.25157967 -0.13231403  0.22705589 -0.14291544
  0.19539687 -0.19493577 -0.20509776 -0.19503365  0.16361083 -0.09872796
 -0.24974737  0.05278531  0.31608379 -0.17115077  0.00947249 -0.20871019
 -0.20496717  0.20510665 -0.05930931 -0.32422054 -0.01459741  0.10232213
 -0.37765491 -0.60824782 -0.15216665 -0.15951824  0.00092883  0.37195647
  0.00309798  0.14055009 -0.12534742  0.11107087 -0.07963439 -0.16241731
  0.30570763  0.04982061 -0.31025037  0.1531795   0.03445475 -0.06693215
 -0.39165884  0.17859174 -0.26240471  0.06031885  0.04702733  0.27970684
  0.16961427  0.01772522  0.15133622 -0.13932395  0.36123252 -0.08790219
 -0.17586106 -0.03695906  0.1025423  -0.00863175 -0.28313485 -0.18812416
  0.19863234 -0.18261926 -0.00105999  0.

345000 全集中
[ 2.69652426e-01  3.19739848e-01  2.17217594e-01  2.09230632e-01
 -8.13427791e-02 -1.71183854e-01  2.45731026e-01 -1.38275117e-01
 -4.54721570e-01 -4.69043665e-02 -2.21411392e-01 -1.44595936e-01
  1.25346333e-01 -1.85916334e-01 -2.07520295e-02 -6.63710907e-02
 -1.21850155e-01  2.83550054e-01  1.67460188e-01  1.16857626e-01
  1.64309710e-01 -5.33549674e-02 -6.21330291e-02 -3.10776651e-01
 -8.11493695e-02  9.71580148e-02 -2.16389626e-01  6.56066313e-02
 -1.56841648e-04 -2.38956645e-01  1.60298139e-01 -2.40080565e-01
  1.34809501e-02  2.90666614e-02 -1.56061813e-01  2.19564345e-02
 -2.46854693e-01  6.42269477e-02  2.86897928e-01 -2.33333334e-02
 -1.54674992e-01  9.62306187e-02 -6.05847279e-04 -2.29240462e-01
  1.81168184e-01 -1.85953215e-01 -4.81650941e-02 -1.61888257e-01
  1.82628214e-01 -1.71575472e-01  7.37659857e-02 -2.77790357e-03
  2.19150573e-01  2.00665981e-01 -4.58459139e-01  7.08443718e-03
 -1.86249912e-01  1.74564973e-01  2.12297775e-02  1.83181018e-02
  7.18867853e-

370000 信息素
[-0.11342426  0.1143583  -0.10669447  0.29917464 -0.44429919  0.29578748
  0.0517545   0.03791366 -0.13643382  0.34139165  0.16930325  0.18992932
  0.31885076  0.38072142 -0.01004036 -0.35849544  0.10321847 -0.11043824
  0.21372215  0.03516607 -0.47006592 -0.27153334 -0.10178281  0.14096144
 -0.05024055  0.17011586  0.20934054 -0.26586974  0.21863204 -0.66512465
 -0.0710284  -0.03389516 -0.27206904 -0.09187947 -0.06650071 -0.3267855
  0.06007193  0.25780416  0.0905758  -0.01388753 -0.19104367 -0.17093582
  0.44559604  0.23512726  0.1202412  -0.05374127 -0.08107097  0.31370017
 -0.05541107 -0.15493447  0.21666057 -0.08034396 -0.12017886 -0.12120166
  0.00515725 -0.41486314 -0.06154322 -0.11503883  0.20025204 -0.12050012
  0.39155173  0.23165001  0.24253002  0.57417071 -0.2633653   0.14115423
  0.18253866  0.08640027  0.06163608  0.20778301 -0.37725762  0.06808279
  0.36554706 -0.18903182 -0.02379637 -0.15051632 -0.24409997  0.08429887
  0.11475492 -0.05731768 -0.18082482  0.3

./vectors/part-00046
395000 竞选分享会
[ 2.50835391e-03  1.40406154e-02 -3.19752060e-02 -2.23857332e-02
 -6.33433182e-03  1.19893451e-03  7.77175929e-03 -8.79878085e-03
 -5.71348555e-02 -2.75335908e-02 -3.35219665e-03 -2.62565613e-02
 -3.33579704e-02 -1.92620289e-02  7.57079013e-03 -7.28046447e-02
 -4.54890691e-02  1.16348471e-02  1.75798126e-02 -6.08411841e-02
 -1.64993163e-02  9.91926435e-03 -1.00008305e-02  2.33722646e-02
 -4.22510086e-04  5.62570766e-02  3.99492569e-02 -3.76584753e-03
 -2.37764185e-03 -3.01969657e-03 -8.52657482e-03  1.09918024e-02
  6.77379817e-02  1.38553474e-02 -1.54618807e-02 -3.24921086e-02
 -1.67118143e-02  5.72073739e-03 -8.38219095e-03  9.03241895e-03
 -6.89136386e-02  2.81230286e-02 -1.81925520e-02 -4.19434831e-02
 -2.05841400e-02 -2.69006612e-03 -5.01630865e-02 -1.60267353e-02
  3.03076804e-02 -1.62644181e-02 -2.74232076e-03 -1.17404526e-02
 -1.57964975e-02 -9.76547040e-03 -1.99506088e-04 -6.44126385e-02
 -1.52826600e-03  4.87711392e-02  2.13882551e-02 -3.3148

./vectors/part-00049
420000 国家排行
[-0.07454339  0.05578781 -0.05173967 -0.25399303 -0.15416242  0.02660158
 -0.11558647 -0.09339747 -0.04644302  0.01694079  0.09946847 -0.06784563
  0.04364803  0.04839024 -0.04174808  0.05652464  0.01360054 -0.03121001
  0.01966956  0.07260293 -0.17077343  0.10004734 -0.04023807  0.10740495
  0.02469989  0.00664217  0.05542055  0.04621375 -0.02836449  0.05253482
  0.07048273 -0.11869108  0.0511185   0.09496181  0.00786052  0.06196095
 -0.03804665 -0.05850563 -0.2403976  -0.04396224 -0.07653613  0.01970461
  0.08788162 -0.04026674  0.14568648  0.15511814  0.08101601 -0.06973758
  0.1038638   0.02795953  0.03415587  0.1693002  -0.01407301  0.00300737
 -0.06522669 -0.19495468 -0.14781567  0.11023258  0.06920879 -0.06195948
  0.15210779 -0.13427071  0.23663002  0.0170088  -0.03154328 -0.12739669
 -0.01534234  0.15768608 -0.05377506  0.07431237  0.13222863 -0.03676866
  0.22719228  0.13168992  0.12996693  0.0531381   0.13112083 -0.05634752
  0.15211579  0.04

In [28]:
### 2 ### load topic2tag map
categoryMap = defaultdict(list)
topicMap = {}
with open ('./data/topic2tags.csv','r') as f:
    for line in f.readlines():
        try:
            category, topic, tags = line.strip().split('\t')
            tags = tags.split(',')
            categoryMap[category].append(topic)
            topicMap[topic] = tags
            print(category, topic)
            print(tags)
        except Exception as e:
            continue

社会 社会热点
['社会热点', '新闻', '時政', '老梁观世界', '国际新闻', 'news', '1818黄金眼', '国际热点', '道德星空', '訪談', '特朗普', '川普', 'yitiaotv', '報道', 'ニュース', 'News', 'trump', 'NHK', '國際新聞', '海峡两岸', '国际热点']
社会 真实案件
['真实案件', '天网恢恢', '探案', '警法目录BTV', '现场说法BTV', '法治进行时BTV', '法治北京BTV', '社会与法', 'CCTV社会与法', 'CCTV法制', '法律講堂', 'CCTV社會與法', 'CCTV12', '案件']
社会 奇闻怪事
['奇闻怪事', '靈異事件', '灵异事件', '鬼', '都市传说', '都市伝説', '身心靈', '奇闻', '都市傳說', '灵异', '悬案', '詭異', 'きまぐれクック', '怪談', '侦探', '怖い話', '未解之謎']
社会 正能量
['正能量', '励志故事', '人生励志', '抖音正能量', '中文正能量', '社会正能量', '社會正能量']
社会 人物
['人物', '一条视频', '一條視頻', '好人故事', '杨澜访谈录', 'yitiao', '感动中国', '感人故事', 'yitiaotv']
社会 社会实验
['社会实验', '大树君', 'social experiment', '社會實驗', '中国街头测试', '大樹君']
社会 时评
['时评', '時政', '时事', '時事', '老梁', '特朗普', '點評', '中国经济', '时政', '雑談']
社会 交通事故
['交通事故', '车祸', '車禍', '交通事故', '中国交通事故', '交通事故合集', '交通意外', '香港交通事故', '大陆交通事故']
社会 灾害
['灾害', '地震', '台風', '海啸', '台风', '龙卷风']
影视 短片
['短片', '短片电影', 'short film', 'animated short films', '微电影', 'animated shorts', '科幻短片', '情景剧', '微電影']
影视 剪辑片段
['剪辑片段', '电影剪辑', '

# functions

In [30]:
### 3 ### vectors innere product
def search_vectors(user_vec, count, index):
    #暴力计算返回内积结果最大的向量
    ids = index[0]
    vectors = index[1]
    tt1 = time.time()
    products = np.inner(user_vec, vectors) #numpy求user_vec 和 item_vecs的内积
    tt2 = time.time()
    # products_id = sorted(range(len(products)), key=lambda x:products[x], reverse=True) #返回排序后的index
    products_id = np.argsort(-products)[:count]
    #ind = np.argpartition(products, -count)[-count:]
    #products_id = ind[np.argsort(products[ind])]
    tt3 = time.time()
    result = [ids[i] for i in products_id] #返回top count条向量的cid
    tt4 = time.time()
    print('inner product cost:%s, sort cost:%s, return cost:%s ' % (int((tt2-tt1)*1000), int((tt3-tt2)*1000), int((tt4-tt3)*1000)))
    print('topN results : ',result)
    return result

def Traditional2Simplified(sentence):
    sentence = Converter('zh-hans').convert(sentence)
    return sentence

def tag_preprocess(tag):
    #tag格式预处理
    sentline = Traditional2Simplified(tag)
    sentline = sentline.replace('\"', '').replace('#', '').replace('”', '').replace('“', '').replace('、', ',')
    sentline = sentline.lower()
    return sentline

# 计算每个主题的向量

In [29]:
### 5 ### 获取每个主题所对应的向量
topicVecMap = {}
emptyTopics = []
emptyTags = []
for topic,tags in topicMap.items():
    tmpTags = [tag_preprocess(tag) for tag in tags]
    tmpVecs = []
    for tag in tmpTags:
        try:
            vec = tagVec[tagName.index(tag)]
            tmpVecs.append(vec)
        except Exception as e:
            emptyTags.append(tag)
            continue
    if len(tmpVecs) == 0:
        emptyTopics.append(topic)
        continue
    tmpVecs = np.array(tmpVecs, dtype=np.float64)
    tmpVecs = np.sum(tmpVecs, axis=0) / len(tmpVecs)
    topicVecMap[topic] = tmpVecs
    

In [34]:
for t in emptyTopics:
    print(t, ' : ',topicMap[t])

欧美综艺  :  ['"欧美综艺', 'Simon Cowell', "America's"]
欧美娱乐  :  ['"欧美娱乐', 'etonline', 'et online', 'Entertainment Tonight', 'celebrity', 'Heidi']
r&b  :  ['r&b', 'Usher', 'Chinese R&B', 'Taiwan R&B', 'Ｒ＆Ｂ']


In [58]:
topic_id = list(topicVecMap.keys())
topic_vec = [topicVecMap[k] for k in topicVecMap.keys()]
topic_index = [topic_id, topic_vec]

# DEMO评测

In [52]:
#获取测试样本
import json
testData = []
with open ('./data/candidates_new_test.txt','r') as f:
    for line in f.readlines():
        try:
            vid, title,tags = line.strip().split('\t')
            testData.append([vid,title,json.loads(tags)])
        except:
            continue
        

In [54]:
import random
random.shuffle(testData)

In [60]:
validData = testData[:100]
validResult = []
invalidResult = []
for item in validData:
    tmpVecs = []
    tmpTags = [tag_preprocess(i) for i in item[2]]
    for tag in tmpTags:
        try:
            vec = tagVec[tagName.index(tag)]
            tmpVecs.append(vec)
        except Exception as e:
            continue    
    if len(tmpVecs) == 0:
        invalidResult.append(item)
        continue
    tmpVecs = np.array(tmpVecs, dtype=np.float64)
    tmpVecs = np.sum(tmpVecs, axis=0) / len(tmpVecs)   
    tmpResult = search_vectors(tmpVecs, 5, topic_index)
    validResult.append([item[0],item[1],item[2],tmpResult])
        
        

inner product cost:0, sort cost:0, return cost:0 
topN results :  ['烘焙', '美食教程', '甜品', '创意料理', '厨具']
inner product cost:0, sort cost:0, return cost:0 
topN results :  ['影视解说', '影评', '美剧', '大陆电影', '剪辑片段']
inner product cost:0, sort cost:0, return cost:0 
topN results :  ['公务员', '柔道', '建筑', '自行车', '装修']
inner product cost:0, sort cost:0, return cost:0 
topN results :  ['台球', '泰国行', '玉石', '法语', '古董']
inner product cost:0, sort cost:0, return cost:0 
topN results :  ['游戏解说', '手游', '历史故事', '游戏集锦', '游戏点评']
inner product cost:0, sort cost:0, return cost:0 
topN results :  ['韩语', '日语', '韩国行', '海外生活', '汉语']
inner product cost:0, sort cost:0, return cost:0 
topN results :  ['柔道', '书法', '武术', '军情', '中国象棋']
inner product cost:0, sort cost:0, return cost:0 
topN results :  ['汉语', '早教', 'K12', '英语', '日语']
inner product cost:0, sort cost:0, return cost:0 
topN results :  ['柔道', '医美', '书法', '牙科', '日本娱乐']
inner product cost:0, sort cost:0, return cost:0 
topN results :  ['棒球', '柔道', '相扑', '高尔夫球', '中国象棋

In [69]:
validResult

[['6689731591625580544',
  '芒果夏洛特蛋糕,夏日芒果系列,#142【明聪Leo】',
  ['夏洛特', '芒果蛋糕', '手指餅乾', '芒果夏洛特蛋糕'],
  ['烘焙', '美食教程', '甜品', '创意料理', '厨具']],
 ['6701324239964540928',
  '陨石群轰击地球，穹顶之下人们安然无恙，外星人的庇护？科幻悬疑剧《穹顶之下》S3-7剧情详解',
  ['电视剧解说', '科幻', '悬疑', '经典美剧', '穹顶之下', '全能打卡挑战', '神奇作品挖掘挑战'],
  ['影视解说', '影评', '美剧', '大陆电影', '剪辑片段']],
 ['6704568093077475328',
  '奥冈电気工事\u3000株式会社',
  ['奥岡電気', '三重', 'いなべ', '配電線', '一般電気', '非鉄金属加工', '未経験', '求人', '高所作業'],
  ['公务员', '柔道', '建筑', '自行车', '装修']],
 ['6674872586646392832',
  '【人生资本论】贸易战没有打垮日本，是什么导致其经济泡沫破裂？',
  ['视知',
   '脑洞电影院',
   '视知百科',
   '视知车学院',
   '人生资本论',
   '女子力科学社',
   '本格男士',
   '喵客帝国',
   '次元光线',
   '视知资本论',
   '视知医学说',
   'cos不累嘟昂',
   '脑子有洞'],
  ['台球', '泰国行', '玉石', '法语', '古董']],
 ['6702215379639144448',
  '百里玄策,王者荣耀,实战,|,荣耀局96星排位赛,|,百里玄策乱杀一切！！［草莓］|,Honor,Of,Kings,Gameplay',
  ['百里玄策',
   '王者荣耀',
   '王者荣耀 百里玄策',
   '百里玄策 王者荣耀',
   '百里玄策 王者荣耀 实战',
   '百里玄策实战',
   '百里玄策 实战',
   '王者荣耀 百里玄策 实战',
   '国服百里玄策',
   '国服百里玄策出装',
   '国服百里玄策铭文',
   '百里玄策怎么玩',
   '排位赛百里

In [63]:
#输出测试结果到本地文件
with open ('testData.csv','w') as f:
    f.write('vid'+'\t'+'title'+'\t'+'original tags'+'\t'+'predict topics'+'\n')
    for item in validResult:
        f.write(item[0]+'\t'+item[1]+'\t'+str(item[2])+'\t'+str(item[3])+'\n')

In [68]:
emptyTopics

['欧美综艺', '欧美娱乐', 'r&b']

In [67]:
#保存每个主题对应的向量
import pickle
file = open('./data/topicVectors.pkl','wb')
pickle.dump(topic_index,file)
file.close()

# 主题的tags补全

In [70]:
appendResults = {}
for key in topicMap.keys():
    if len(topicMap[key]) != 0:
        print('******'*5)
        print(key, topicMap[key])
        print('******'*5)
        tmpTags = topicMap[key]
        tmpVecs = []
        
        for tag in tmpTags:
            try:
                sentline = Traditional2Simplified(tag)
                sentline = sentline.replace('\"', '').replace('#', '').replace('”', '').replace('“', '').replace('、', ',')
                sentline = sentline.lower()         
                vec = tagVec[tagName.index(sentline)]
                tmpVecs.append(vec)
            except Exception as e:
                print(e)
                continue
                
        if len(tmpVecs) == 0:
            continue
        tmpVecs = np.array(tmpVecs, dtype=np.float64)
        tmpVecs = np.sum(tmpVecs, axis=0) / len(tmpVecs)
        #print('vector : ', tmpVecs)    
        tmpResult = search_vectors(tmpVecs, 30, indexs)
        #print(result)
        appendResults[key] = tmpResult

******************************
社会热点 ['社会热点', '新闻', '時政', '老梁观世界', '国际新闻', 'news', '1818黄金眼', '国际热点', '道德星空', '訪談', '特朗普', '川普', 'yitiaotv', '報道', 'ニュース', 'News', 'trump', 'NHK', '國際新聞', '海峡两岸', '国际热点']
******************************
'news' is not in list
'yitiaotv' is not in list
'ニュース' is not in list
'news' is not in list
'trump' is not in list
'nhk' is not in list
inner product cost:2077, sort cost:49, return cost:0 
topN results :  ['国际热点', '国际新闻', '时政', '热点新闻', '军事新闻', '热点时局', '军事节目', '中国军事', '世界新闻', '时事热点', '热点互动', '亚洲新闻', '海峡两岸', '政论', '国内新闻', '时政新闻', '时事新闻', '要闻', '时事点评', '新闻报道', '张召忠', '时事政治', '新闻解读', '紧张局势', '地缘政治', '早间新闻', '台海', '美洲新闻', '军事科技', '民生民泰']
******************************
真实案件 ['真实案件', '天网恢恢', '探案', '警法目录BTV', '现场说法BTV', '法治进行时BTV', '法治北京BTV', '社会与法', 'CCTV社会与法', 'CCTV法制', '法律講堂', 'CCTV社會與法', 'CCTV12', '案件']
******************************
'现场说法btv' is not in list
'法治进行时btv' is not in list
'法治北京btv' is not in list
'cctv12' is not in list
inner product cost:397, sort

inner product cost:360, sort cost:44, return cost:0 
topN results :  ['电影信息', '电影杂志', '短片电影', '排名电影', '最新电影', '讲电影', '说电影', '好看电影', '深入电影', '热门电影', '好看解说', '深度电影', '五分钟看电影', '电影解说推荐', '十分钟看电影', '亮哥讲电影', '豆瓣电影', '三分钟看电影', '神速完剧', '牛叔', '内地电影', '大陆电影', '分分钟看完电影', '电影吐槽', '几分钟看完电影', '电影评论', '几分钟看完', '几分钟说电影', '剧情电影', '中国电影']
******************************
大陆电视剧 ['大陆电视剧', '中国电视剧', '武媚娘传奇', '大陆电视剧', '陳情令', '守卫者浮出水面Noah', '浪花一朵朵Noah', '我的前半生Noah', '青云志Noah', '河山Noah', '莫斯科行动Noah', '你好乔安Noah', '法医秦明Noah', '金牌投资人Noah', '好久不见Noah', '知否知否应是绿肥红瘦Noah', '热爱Noah', '我站在桥上看风景Noah', '爱情进化论Noah', '大陆剧', '网剧', '延禧攻略', 'chinese drama', 'TV drama program', '我在北京等你Noah', '連續劇', '谍战', '马跃', 'Ma Yue']
******************************
'知否知否应是绿肥红瘦noah' is not in list
'chinese drama' is not in list
'tv drama program' is not in list
'ma yue' is not in list
inner product cost:370, sort cost:43, return cost:0 
topN results :  ['箭在弦上noah', '浪花一朵朵noah', '守卫者浮出水面noah', '莫斯科行动noah', '都挺好noah', '我的前半生noah', '惊蛰noah', '好看的

inner product cost:357, sort cost:44, return cost:0 
topN results :  ['白凯南', '方清平', '马跃', '陈怡', '黄大炜', '宋小宝', '欢乐喜剧人', '心灵点滴', '心灵小品', '励志小语', '陈玉珠', '辽宁卫视', '励志的话', '心灵勇气', '心灵钥匙', '危险心灵', '心灵舒果', '心灵飨宴', '心灵小语', '心灵语坊', '温馨励志小语', '励志文章', '欢乐喜剧人第三季', '心灵投手', '赵本山', '开心麻花', '心灵享宴', '心旅行', '黄景昭心灵飨宴', '小岳岳']
******************************
广播 ['广播', '電台', '广播剧', '文艺广播', '鬼故事朗读', '海盗电台', '午夜伤心电台']
******************************
'海盗电台' is not in list
inner product cost:355, sort cost:43, return cost:0 
topN results :  ['评书', '有声小说', '广播剧', '有声读物', '2020动漫', '有声', '豪门恩怨', '小说推荐', '听书', '有声故事', '妖魔鬼怪', '恐怖有声小说', '鬼叔怪谈', '朗读', '小说改编', '胆小勿点', '漫漫声工坊', '传闻', '毛骨悚然', '坑王驾到', '网上电台', '鬼叔', '灵异小说', '民间传说', '中文电影', '音频节目', '言情小说', '网上节目', '恐怖小说', '言情']
******************************
内地娱乐 ['内地娱乐', 'ICE', '李晨', '杨超越', '周杰', '肖战', '娛樂百分百', '金莎', '王一博', '蔡徐坤', '范冰冰', '王源', '朱一龙', 'TFBOYS', '杨紫', '刘芸', '每日文娱播报BTV', '张艺兴', '张含韵', '李斯丹妮', '易烊千玺', '金晨', '杨幂', '郁可唯', '贾玲', '吴宣仪', '李维嘉', '任嘉伦', '赵丽颖', '博君一肖'

inner product cost:365, sort cost:43, return cost:0 
topN results :  ['搞笑段子', '超级搞笑', '抖音笑话', '冷笑话', '爆笑小品', '爆笑视频', '美女搞笑', '很好笑', '段子', '搞笑合集', '最新搞笑视频', '猪屁登', '摩登大自然', '抖音集锦', '抖音最火动画', '超好笑', '搞笑抖音', '抖音搞笑视频', '超级搞笑视频', '胡成功抖音', '太逗了', '唐朝发明家', '抖音搞笑', '小品一家人', '波波脱口秀', '搞笑小品', 'tok推介', '开心的笑', '捏他', '抖音推介']
******************************
搞笑短片 ['"搞笑短片', 'お笑い']
******************************
'お笑い' is not in list
inner product cost:347, sort cost:42, return cost:0 
topN results :  ['搞笑短片', '爆笑短片', '超级搞笑', '玩笑', '爆笑小品', '很好笑', '抖音笑话', '爆笑短剧', '爆笑集锦', '忍笑挑战', '搞笑短剧', '搞笑合集', '冷笑话', 'andy老师', '蕾蕾tv', '美女搞笑', '搞笑图片', '搞笑抖音', '超好笑', '抖音最新搞笑', '搞笑动画', '搞笑短视频', '搞笑的视频', '太逗了', '笑话集', '热门视频', '波波脱口秀', '最新搞笑', '最新搞笑视频', '幽默短片']
******************************
恶搞 ['恶搞', '搞怪', 'funny pranks', 'prank', '恶整', '整人', '惡作劇', 'best pranks', '惡整', '整蛊', 'kuso', 'hacks', 'pranks', 'lifehacks', 'trucos', '恶作剧', '黃氏兄弟', 'Prank', 'beauty hacks', '整蛊', 'wackyboys', 'PRANK', 'tricks', '面白', '狠愛演', 'parody']

inner product cost:349, sort cost:43, return cost:0 
topN results :  ['洛丽塔', '妖娆', '穿衣', '御姐', '制服诱惑', '可爱的男孩子', '麦霸', '热吻', '尬聊电影', '服装搭配', '东方美女', '时尚穿搭', '百褶裙', '少女感', '短裙', '清纯', '男扮女装', '内地电影', '黑丝', '多余和毛毛姐', '服饰搭配', '到达巅峰', '女仆装', '丝袜美腿', '甜宠', '温婉', '宅男杀手', '美妆达人', '腹黑', '好嗨哦']
******************************
JK制服 ['JK制服', 'jk制服', '制服のマネキン', '水手服', 'セーラー服', '学生服']
******************************
inner product cost:355, sort cost:43, return cost:0 
topN results :  ['服装搭配', '时尚穿搭', '可爱的男孩子', 'jk制服', '服饰搭配', '百褶裙', '穿搭分享', '短裤', '短裙', '服饰', '洋装', '高跟', '学生党', '就是亚军', '穿衣', '穿搭技巧', '女装', '上衣', '女装子', '裙子', '吊带', '小个子', '黑丝袜', '御姐', '搭配', '洛丽塔', '文胸', '微胖', '高跟鞋', '夏季穿搭']
******************************
女仆装 ['女仆装', 'はじめしゃちょー', '私房写真', '黑丝', '短裙', '高跟', '紧身裤', '丝袜美腿', '东方美女', '惹火身材', '情趣内衣']
******************************
'はじめしゃちょー' is not in list
inner product cost:362, sort cost:44, return cost:0 
topN results :  ['丝袜美腿', '高跟', '黑丝', '连裤袜', '紧身裤', '丝袜', '黑色丝袜', '短裙', '东方美女', '美腿', '美足

...

inner product cost:380, sort cost:43, return cost:0 
topN results :  ['美国买房', '美国房产', '美国豪宅', '美国房地产', '休斯顿房产', '美国房产投资', '洛杉矶买房', '美国投资', '尔湾', '尔湾房产', '洛杉矶地产经纪', '尔湾买房', '投资房', '洛杉矶房产', '美国买房注意事项', '尔湾地产经纪', '卖房', '学区房', '休斯顿买房', '海外地产', '自住房', '尔湾房产经纪人', '美国看房', '美国房产经纪', '洛杉矶房产经纪', '尔湾豪宅', '房产经纪人', '休斯顿置业', '看房日记', '看房视频']
******************************
期货 ['期货', '期貨', '原油', '期货交易', '比特币期货', '股指期货', '原油期货', '期货损耗', '期货投机', '币圈期货合约', '外汇美股期货']
******************************
inner product cost:383, sort cost:43, return cost:0 
topN results :  ['投顾', '程式交易', '期指', '牛熊证', '技术指标', '台指期', '道琼斯', '分析师', '期货交易', '飙股', '台湾股市', 'k线', '量化交易', '解盘', '选择权', '陆股', '期权', '恒生指数', '外汇交易', '上证指数', 'k线图', '空头', '期指教学', '股票期货', '波浪理论', '台指', '当冲', '外汇保证金', '期货', '外汇市场']
******************************
汽车评测 ['"汽车评测', '萝卜报告', '賓士', '汽车之家', 'トヨタ', 'jdm', '车评', 'audi', 'jyp', 'chevy', 'Mercedes-Benz', 'TOYOTA', 'audi', 'porsche', '跑車']
******************************
'トヨタ' is not in list
'jdm' is not in lis

'us navy' is not in list
inner product cost:390, sort cost:44, return cost:0 
topN results :  ['狙击枪', '央视军事', '军事新闻', '实况直播', '狙击手', '手雷', '今日热搜', '武器介绍', '三级甲', '无限法则', '枪械', '步枪', 'cctv军事', '挑战任务', '枪战游戏', '军情', '军备', '军事报道', '战列舰', '和平精英挑战', 'g港', '军事科技', '生死狙击', '核武器', '信号枪', '驱逐舰', '军事频道', '八路军', '空投', '军演']
******************************
演讲 ['演讲', 'TED解说', 'TED出品', '蒋勋 演讲', '蒋勋2019演讲', '演讲技巧', '内涵演讲', '马云演讲', '成功学演讲', 'ted演讲', '巴菲特演讲']
******************************
'蒋勋 演讲' is not in list
inner product cost:388, sort cost:45, return cost:0 
topN results :  ['xiaoy解说', 'ted解说', 'ted出品', 'sky来了', 'dota2解说', '英雄联盟解说', '也好爱lol', '魔兽争霸', '也好脱坑指南', 'lol解说', '丸子解说', '小漠解说', '林剑锋', '教主英雄联盟', '也好解说', 'lol英雄联盟', '战队粤语', '马云演讲', '解说小漠', 'jy解说', '马云创业', '战队', 'tvb粤语op', '思维格局', '大树君', '好文', '商业思维', '幪面超人粤语', '章鱼丸718', '晓东']
******************************
职场 ['职场', 'インタビュー', '職場', '職人', '時間管理', '求职', '事業', '職業', '机器学习', '深度学习', '自我提升']
******************************
'インタビュー' is not in list
in

inner product cost:373, sort cost:44, return cost:0 
topN results :  ['有声读物', '有声小说', '听书', '深夜读书', '音频节目', '读书心得', '评书', '每天听本书', '言情小说', '美文', '陪你读书', '听小说', '速读', '有声书', '从心阅读', '儿童故事', '读书分享', '听书世界', '小说推荐', '名著', '说书', '读册', '手绘本', '阅读', '故事汇', '樊登读书', '畅销书', '有声故事', '故事分享', '中文故事']
******************************
英语 ['"英语', '英语学习初级 初级英语', '英语学习', '阿滴英文', '英語学習', '阿滴', '英语单词学习', '学英文', 'learn']
******************************
'英语学习初级 初级英语' is not in list
'learn' is not in list
inner product cost:384, sort cost:44, return cost:0 
topN results :  ['英语词汇', '生活英语', '实用英语', '英语单词', '旅游英语', '留学英语', '英语发音', '初级英语', '常用英文', '美式英语', '初级英文', '英语听力', '英语课程', '英语文法', '英语口语', '英文文法', '学习英文', '英语短语', '英语单词学习', '基础英文', '旅行英文', '英语会话', '学英语会话', '英语语法', '学习英语', '日常英语', '基础英语', '英语句子', '英语课', '英文单字']
******************************
韩语 ['韩语', '生活韩语', '学习韩语', '韩语学习', '韩语课堂', '韩语材料', '韩语自学', '韩语发音', '初级韩语', '自学韩语', '韩语入门']
******************************
inner product cost:382, sort cost:43, return cost:

inner product cost:371, sort cost:44, return cost:0 
topN results :  ['鉴宝', '华山论鉴', '翡翠功效', '翡翠观音', '帝王绿', '缅甸翡翠', '翡翠知识', '翡翠原石', '翡翠拍卖', '玻璃种', '翡翠手镯', '翡翠a货', '翡翠直播', '翡翠挂件', '翡翠饰品', '收藏节目', '冰种翡翠', '翡翠赌石', '珠宝投资', '华豫之门', '墨翠', '天然翡翠', '赌石', '缅甸玉', '宝石鉴定', '翡翠戒面', '祖母绿', '翡翠摆件', '奢饰品', '翡翠吊坠']
******************************
艺术 ['艺术', '布袋戲', '艺术', '艺术家', '人文艺术', '中国艺术', '艺术之美', '当代艺术', 'Artist']
******************************
'artist' is not in list
inner product cost:397, sort cost:44, return cost:0 
topN results :  ['临摹', '人文艺术', '绘画教程', '绘画教学', '山水画', '国画', '书法', '行为艺术', '速画', '儿童画', '绘画过程', '当代艺术', '水彩教程', '书法教学', '学画画', '画画教学', '工笔', '油画棒', '速写', '简笔画', '水墨', '水彩技法', '木颜色', '水彩风景', '板绘', '水墨画', '毛笔字', '学水彩', '原住民族广播电台', '如何画']
******************************
说书 ['说书', '评书', '說書人', '書評', '四大名著', '蒋勋 红楼梦', '北京评书大会BTV']
******************************
'蒋勋 红楼梦' is not in list
'北京评书大会btv' is not in list
inner product cost:378, sort cost:43, return cost:0 
topN results :  ['评书', '有声读物'

inner product cost:380, sort cost:45, return cost:0 
topN results :  ['益智游戏', '儿童游戏', '玩的宝宝的生活', '小制作', '益智玩具', '儿童玩具', '儿童动画', '亲子玩具', '嘿基尼', '儿童故事', '教育漫画', '玩玩具', '幼儿教育', '儿童视频', '教育孩子', '植物大战僵尸', '乐高玩具', '小猪佩奇', '自制玩具', '过家家游戏', '伶可家族', '积木玩具', '波力', '中文卡通', '中文动漫', '儿童启蒙', '儿童节目', '基尼', '儿童歌曲', '汽车玩具']
******************************
儿童动画 ['"儿童动画', '迷你世界动画片', '小猪佩奇', 'videos for kids', '佩佩豬', 'Cartoon', '动画片', 'cartoons', '巧虎', '宝宝巴士', 'cartoons for kids', 'kids']
******************************
'videos for kids' is not in list
'cartoon' is not in list
'cartoons' is not in list
'cartoons for kids' is not in list
'kids' is not in list
inner product cost:389, sort cost:44, return cost:0 
topN results :  ['翼龙妈咪', '儿童动画', '小猪佩奇', '宝宝巴士', '贝瓦儿歌', '幼儿英语', '玩具故事', '英语启蒙', '巧虎', '儿童视频', '儿歌童谣', '英文儿歌', '儿童启蒙', '经典童谣', '粉红猪小妹', '鲨鱼一家', '儿歌大全', '碰碰狐', '儿歌三百首', '玛莎与熊', '佩佩猪', '小猪一家亲', '粉红猪小妹玩具', '幼儿教育', '儿童歌曲', '英文启蒙', '经典儿歌', '佩奇', '刷牙歌', '智慧卡通']
******************************
童话故事 ['童话故事', '

In [79]:
#保存appendResults
with open ('./data/tagAppendix.txt','w') as f:
    f.write('主题' + '\t' + '补充tags' + '\n\n')
    for k in appendResults.keys():
        v = appendResults[k]
        f.write(str(k) + '\t\t' + str(v) + '\n\n')

# 接入Bert模型补全找不到向量的主题